<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

**Таблица churn содержит следующие столбцы:**

* `RowNumber` — индекс строки в данных
* `CustomerId` — уникальный идентификатор клиента
* `Surname` — фамилия
* `CreditScore` — кредитный рейтинг
* `Geography` — страна проживания
* `Gender` — пол
* `Age` — возраст
* `Tenure` — сколько лет человек является клиентом банка
* `Balance` — баланс на счёте
* `NumOfProducts` — количество продуктов банка, используемых клиентом
* `HasCrCard` — наличие кредитной карты
* `IsActiveMember` — активность клиента
* `EstimatedSalary` — предполагаемая зарплата
* `Exited` — факт ухода клиента (целевой признак)

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [2]:
churn = pd.read_csv('/datasets/Churn.csv')
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**В столбце tenure есть NaN, посмотрим значения:**

In [4]:
churn['Tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

In [5]:
churn['Tenure'] = churn['Tenure'].fillna(0)

**Заполним все пропуски нулями, это лучше, чем дропать 10% данных, т.к. все остальные признаки заполнены адекватно, заполнять медианным значением не выглядит правильным**

In [6]:
churn['Tenure'].value_counts()

0.0     1291
1.0      952
2.0      950
8.0      933
3.0      928
5.0      927
7.0      925
4.0      885
9.0      882
6.0      881
10.0     446
Name: Tenure, dtype: int64

**Переведем столбцы из camel case'а в snake case, так привычнее**

In [7]:
churn.columns = [i.lower() for i in churn.columns]

In [8]:
churn.rename(columns={"rownumber":"row_number",
                      "customerid":"customer_id",
                      "creditscore":"credit_score",
                      "numofproducts":"num_of_products",
                      "hascrcard":"has_cr_card",
                      "isactivemember":"is_active_member",
                      "estimatedsalary":"estimated_salary"
                      }, inplace=True)

In [9]:
churn.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

**Для дальнейшего исследования нам не нужны столбцы row_number, customer_id и surname, дропнем их**

In [10]:
churn = churn.drop({"row_number", "customer_id", "surname"}, axis=1)

In [11]:
churn.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


## Исследование задачи

**Преобразуем категориальные признаки техникой One Hot Encoding**

In [12]:
churn_ohe = pd.get_dummies(churn, drop_first=True)

**Выделим целевой признак и разобьем данные на выборки**

In [13]:
features = churn_ohe.drop('exited', axis=1)
target = churn_ohe['exited']

In [14]:
features_train, features_valid, target_train, target_valid = train_test_split (
    features, target, test_size=0.4, random_state=0)

features_valid, features_test, target_valid, target_test = train_test_split (
    features_valid, target_valid, test_size=0.5, random_state=0)

**Приведем признаки к одному масштабу**

In [15]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card', 'is_active_member', 'estimated_salary']

In [16]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

/tmp/ipykernel_67/627840052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [17]:
features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
7809,-0.744432,0.400729,-0.810507,1.123796,-0.936474,0.642724,0.966881,1.702726,0,1,1
5279,0.009441,0.111996,0.792130,0.401011,-0.936474,0.642724,0.966881,0.422349,0,1,0
3279,0.009441,-0.176737,-1.451562,-1.199501,0.798808,0.642724,-1.034253,-0.544388,0,1,1
8984,-0.062849,-1.331668,-1.451562,1.578702,-0.936474,0.642724,-1.034253,-0.997245,0,0,1
8466,-0.393313,-0.272981,1.433185,0.892225,-0.936474,-1.555877,-1.034253,-0.292067,0,0,1


In [18]:
features_train.shape

(6000, 11)

**Исследуем баланс классов**

In [19]:
target_train.value_counts(normalize=True)

0    0.7985
1    0.2015
Name: exited, dtype: float64

**Видим крайне сильный дисбаланс классов, обучим модели без учета баланса и посмотрим на результаты**

**Начнём с регрессии:**

In [20]:
model = LogisticRegression(solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:,1]

print(f1_score(target_valid, predicted_valid))
print(roc_auc_score(target_valid, probabilities_one_valid))

0.3286219081272085
0.7758674643350206


**Довольно низкое значение f1, посмотрим древо решений:**

In [21]:
best_f1 = 0
best_depth = 0
best_auc_roc = 0

for i in range(1,11):
    model = DecisionTreeClassifier(random_state=0, max_depth=i)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:,1]
    
    f1 = f1_score(target_valid, predicted_valid)
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth = i
        best_auc_roc = auc_roc
        
print(best_f1)
print(best_depth)
print(best_auc_roc)

0.5930851063829788
8
0.8423853351741064


**Результат уже гораздо лучше, более того он предварительно проходит порог в 0.59. Auc_roc_score тоже довольно высокий. Далее обучим модель случайного леса:**

In [22]:
best_f1 = 0
best_est = 0
best_depth = 0
best_auc_roc = 0

for i in range (1,15):
    for j in range (1,101,10):
        model = RandomForestClassifier(random_state=0, n_estimators=j, max_depth=i)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:,1]
    
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        f1 = f1_score(target_valid, predicted_valid)
        
        if f1 > best_f1:
            best_f1 = f1
            best_est = j
            best_depth = i
            best_auc_roc = auc_roc
            
print(best_f1)
print(best_est)
print(best_depth)
print(best_auc_roc)

0.6289120715350224
61
12
0.8683126246356803


**Результат получился еще лучше**

## Борьба с дисбалансом

In [23]:
model = LogisticRegression(solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:,1]

print(f1_score(target_valid, predicted_valid))
print(roc_auc_score(target_valid, probabilities_one_valid))

0.5047372954349698
0.7780947998159228


**Результат по обоим метрикам вырос: f1 значительно увеличился, auc_roc почти не изменился**

In [24]:
best_f1 = 0
best_depth = 0
best_auc_roc = 0

for i in range(1,11):
    model = DecisionTreeClassifier(random_state=0, max_depth=i, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:,1]
    
    f1 = f1_score(target_valid, predicted_valid)
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    
    if f1 > best_f1:
        best_f1 = f1
        best_depth = i
        best_auc_roc = auc_roc
        
print(best_f1)
print(best_depth)
print(best_auc_roc)

0.601145038167939
7
0.830377358490566


**Результат почти не изменился: f1 незначительно увеличился, auc_roc незначительно уменьшился**

In [25]:
best_f1 = 0
best_est = 0
best_depth = 0
best_auc_roc = 0

for i in range (1,15):
    for j in range (1,101,10):
        model = RandomForestClassifier(random_state=0, n_estimators=j, max_depth=i, class_weight='balanced')
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:,1]
    
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        f1 = f1_score(target_valid, predicted_valid)
        
        if f1 > best_f1:
            best_f1 = f1
            best_est = j
            best_depth = i
            best_auc_roc = auc_roc
            
print(best_f1)
print(best_est)
print(best_depth)
print(best_auc_roc)

0.6456140350877194
21
10
0.8648887866237153


**Балансировка веса классов вновь незначительно повлияла на результат. Модель случайного леса показала лучший результат. Посмотрим, улучшится ли результат если использовать метод апсемплинга вместо изменения веса классов.**

In [26]:
def upsample(features, target):
    
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * 4)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * 4)
    
    features_upsampled = shuffle(features_upsampled, random_state=0)
    target_upsampled = shuffle(target_upsampled, random_state=0)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train)

In [27]:
target_upsampled.value_counts()

1    4836
0    4791
Name: exited, dtype: int64

**Теперь, когда баланс классов близок к единице, проверим результаты модели еще раз:**

In [28]:
best_f1 = 0
best_est = 0
best_depth = 0
best_auc_roc = 0

for i in range (1,15):
    for j in range (1,101,10):
        model = RandomForestClassifier(random_state=0, n_estimators=j, max_depth=i)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:,1]
    
        auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
        f1 = f1_score(target_valid, predicted_valid)
        
        if f1 > best_f1:
            best_f1 = f1
            best_est = j
            best_depth = i
            best_auc_roc = auc_roc
            
print(best_f1)
print(best_est)
print(best_depth)
print(best_auc_roc)

0.6467991169977926
91
11
0.876152784169351


**Результат практически идентичен результату аналогичной модели, но с балансом классов. Дополнительно проверим обе модели на тестовой выборке, может быть там будет явный лидер**

## Тестирование модели

In [29]:
model = RandomForestClassifier (random_state=0, n_estimators=21, max_depth=10, class_weight='balanced')
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:,1]

print(f1_score(target_test, predicted_test))
print(roc_auc_score(target_test, probabilities_one_test))

0.6087962962962964
0.8598822579376841


In [30]:
model = RandomForestClassifier (random_state=0, n_estimators=21, max_depth=10)
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:,1]

print(f1_score(target_test, predicted_test))
print(roc_auc_score(target_test, probabilities_one_test))

0.6051813471502591
0.8555852624320254


**Как можно видеть, и при изменении веса классов, и при достижении их баланса результаты получаются практически одинаковыми. Модель случайного леса показала себя хорошо на тестовой выборке, порог пройден.**